In [ ]:
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.qparser import QueryParser
from whoosh.query import *
import csv
import itertools

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta as dt
pd.set_option("display.max_rows",None)
#os.chdir('C:\\Users\\somd7w\\Desktop\\DL_Projects\\preproc_cntr')
cwd = os.getcwd()
data =cwd+'/data'
output=cwd+'/output'

In [ ]:
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = os.listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

In [ ]:
def read_sheet(a):
    try:
        df = pd.read_csv(csv_dict[str(a)], index_col=False)
    except:
        try:
            df = pd.read_csv(csv_dict[str(a)],index_col=False,encoding='latin-1')
        except:
            print('Error Opening CSV File')
    print("Columns of sheet: ",a)
    print(list(df))
    return df

def get_df_stats(df,drop_cols):
    # Drop rows if values are NaN in PAT_ID, CT_SCAN_DATE
    df = df.dropna(subset=drop_cols)
    # Unique PAT_IDs and Given_MRN:
    print("Number of rows in sheet = ",len(df))
    pat_ids = df.PAT_ID.unique()
    print("Number of unique PAT_IDs = ",len(pat_ids))
    mrns = df.GIVEN_MRN.unique()
    print("Number of unique MRNs = ",len(mrns))
   # accs = df.ACC.unique()
   # print("Number of unique ACC = ",len(accs))
    return(df)
    
def filter_duplicates(df,filter_col,sort_order):
    if sort_order == 'ascending':
     #   print('Sorting by ascending order and returning largest value')
        df = df.sort_values(by=[filter_col])
    else:
      #  print('Sorting by descending order and returning smallest value')
        df = df.sort_values(by=[filter_col],ascending=False)
    
    df = df.drop_duplicates('PAT_ID', keep='last')
    return(df)

# Read CSV files from EPIC query, print and store as Dictionary

In [ ]:
csv_list = find_csv_filenames(data)
csv_dict = {}
print("List of Files from EPIC Query:")
for i,csv in enumerate(csv_list):
    print(i+1,': ',csv)
    csv_dict[str(i+1)] = os.path.join(data,csv)

In [ ]:
print(csv_dict)

# Step 1: Read Sheet 2 - Imaging Result

| Field        	| Field description                  	|
|--------------	|------------------------------------	|
| GIVEN_MRN    	| MRN provided                       	|
| NEW_MRN      	| New MRN (if updated)               	|
| PAT_ID       	| Patient Identifier field           	|
| CT_SCAN_DATE 	| Scan completed date provided       	|
| ACC          	| Accession number provided          	|
| STUDYRESULT  	| Result narrrative text             	|
| IMPRESSION   	| Impression notes related to result 	|

In [ ]:
df_2 = read_sheet(2)

In [ ]:
# Drop rows if values are NaN in PAT_ID, CT_SCAN_DATE
drop_cols =['PAT_ID','CT_SCAN_DATE']
df_2 = get_df_stats(df_2,drop_cols)

In [ ]:
df_2['CT_SCAN_DATE'] = pd.to_datetime(df_2.CT_SCAN_DATE)
df_2['CT_SCAN_DATE'] = df_2['CT_SCAN_DATE'].dt.date
df_F = df_2
del(df_2)

In [ ]:
#Visualize Sheet
display(df_F.head(100))

In [ ]:
print('earliest date: ',min(df_F['CT_SCAN_DATE']))

In [ ]:
print('earliest date: ',max(df_F['CT_SCAN_DATE']))

# Step 2: Read Sheet 1 - Height, Weight, BMI sheet 

| Field                  	| Field description        	|
|------------------------	|--------------------------	|
| GIVEN_MRN              	| MRN provided             	|
| NEW_MRN                	| New MRN (if updated)     	|
| PAT_ID                 	| Patient Identifier field 	|
| ACC                    	| Given Accession number   	|
| CT_SCAN_DATE           	| Given completed date     	|
| WEIGHT                 	| Weight (in kg)           	|
| HEIGHT                 	| Height (in cm)           	|
| BMI_FOR_AGE_PERCENTILE 	| BMI for age percentile   	|
| BMI_CALCULATED         	| Calculated BMI           	|

In [ ]:
df_1 = read_sheet(1)

In [ ]:
# Drop rows if values are NaN in PAT_ID, CT_SCAN_DATE
drop_cols =['PAT_ID','CT_SCAN_DATE']
df_1 = get_df_stats(df_1,drop_cols)

In [ ]:
print(len(df_1))
df_1 = filter_duplicates(df_1,'CT_SCAN_DATE','ascending')
print(len(df_1))

In [ ]:
#Visualize Sheet
display(df_F.head(100))

In [ ]:
# Merge dataframes
print("Merge Imaging and BMI Results on PAT_ID and CT_SCAN_DATE and keeping the latest record for duplicates")
df_1['CT_SCAN_DATE'] = pd.to_datetime(df_1.CT_SCAN_DATE)
df_1['CT_SCAN_DATE'] = df_1['CT_SCAN_DATE'].dt.date
df_F = df_F.merge(df_1.drop(['GIVEN_MRN','NEW_MRN','ACC'],axis=1),how='inner', on=['PAT_ID','CT_SCAN_DATE'])


In [ ]:
df_F = filter_duplicates(df_F,'CT_SCAN_DATE','ascending')

In [ ]:
len(df_F)

In [ ]:

df_F = get_df_stats(df_F,drop_cols)

In [ ]:
#Visualize Sheet
display(df_F.head(100))
df_F.to_csv(output+'/corrected_whoosh_data.csv')

# Step 3: Read Sheet 7 - Patient Problem List

| Field          	| Field description                         	|
|----------------	|-------------------------------------------	|
| GIVEN_MRN      	| MRN provided                              	|
| NEW_MRN        	| New MRN (if updated)                      	|
| PAT_ID         	| Patient Identifier field                  	|
| PROBLEM        	| Diagnosis/problem description             	|
| NOTED_DATE     	| Date of diagnosis                         	|
| CODE           	| ICD code associated                       	|
| PROBLEM_STATUS 	| Status of problem active/resolved/deleted 	|
| DATE_OF_ENTRY  	| Date of record entry                      	|

In [ ]:
df_7 = read_sheet(7)
#Visualize Sheet
df_7 = get_df_stats(df_7,['PAT_ID'])

In [ ]:
# Sort by Latest date for problem list
df_7['DATE_OF_ENTRY'] = pd.to_datetime(df_7.DATE_OF_ENTRY)
df_7 = df_7.sort_values(by='DATE_OF_ENTRY',ascending=False)
display(df_7.head(100))

In [ ]:
# Debug
display(df_7[df_7['PAT_ID'].isin(['Z1256768','Z832424'])])
df_debug = df_7[df_7['PAT_ID'].isin(['Z1256768','Z832424'])]


In [ ]:
from data.search_terms_v5 import get_terms
problem_list = get_terms()
# Convert all terms to lower for Whoosh search
problem_list = {k.lower(): [vi.lower() for vi in v] for k, v in problem_list.items()}
display(problem_list)
#print(len(problem_list.keys()))

In [ ]:
import itertools
print(len(problem_list.keys()))
n_searchterms = list(itertools.chain(*problem_list.values()))
print('Number of problems: ', len(n_searchterms))

In [ ]:
# Find number of patients that are in the problem list
print("Number of patients in Master List: ")
print(len(df_F))
print("Number of patients in Problem List: ")
prob_pats = [pat_id for pat_id in df_F.PAT_ID if pat_id in df_7.PAT_ID.values]
print(len(prob_pats))

In [ ]:
df_F["PROBLEM_GROUP"] = None
df_F["PROBLEM_ID"] = None
df_F["PROBLEMS"] = None
pat_ind = df_F.PAT_ID.isin(prob_pats).values
pat_ind_neg =~pat_ind
df_F.loc[pat_ind_neg, 'PROBLEM_GROUP'] = 'no_problem'
df_F.loc[pat_ind, 'PROBLEM_GROUP'] = 'unknown_problem'
display(df_F.head(100))

In [ ]:
len(df_F)

In [ ]:
# Create schema for Problem List search
schema = Schema(mrn=ID(stored=True), content=TEXT(stored=True))

# indexdir is a directory to store the search index
ix = create_in(output+'/whoosh', schema)

writer = ix.writer()

for ind in df_7.index:
    # add a document to search index that is keyed by MRN with the content being the problem list
    #print (ind)
    problem = df_7.loc[ind,'PROBLEM']
    pat_id = df_7.loc[ind,'PAT_ID']
    #print(problem)
    #break
    if pd.isnull(problem):
        #print(problem,type(problem))
        problem = ''
    writer.add_document(
    mrn=pat_id,
    content=problem
    )
writer.commit()

In [ ]:
# Perform Query
with ix.searcher() as searcher:
    # get all of the terms from the problem list and make term objects
    #terms = [Term("content", problem) for problem in itertools.chain.from_iterable(problem_list.values())]
    for key,value in problem_list.items():
        terms = list()
        for problem in value:
            if isinstance(problem, str):
                words = problem.split(' ')
                if len(words) == 1:
                   # print(problem)
                    terms.append(Term("content",problem))
                else:
                    word_t  = list()
                    for word in words:
                        word_t.append(Term("content",word))
                    terms.append(And(word_t))
            else:
                phrase = list()
                for prob in problem:
                    phrase.append(Term("content",prob))
                #print(phrase)    
                terms.append(And(phrase))
  
        # Generates a query that will match cancer or lymphoma or neuroblastoma or ... and so on
        query = Or(terms)

        # terms=True so we can see what matched, limit=None b/c we want all matches
        results = searcher.search(query, terms=True, limit=None)
        print(results.estimated_length())
        print("Group searched", key)
        for hit in results:
          #  print(hit['mrn'])
          #  print([x[1] for x in hit.matched_terms()])
            if hit['mrn'] in prob_pats:
               # print (hit['mrn'], ' ',df_F.loc[df_F['PAT_ID']==hit['mrn'],'PROBLEM_GROUP'].values)
                if df_F.loc[df_F['PAT_ID']==hit['mrn'],'PROBLEM_GROUP'].values[0] == 'unknown_problem':
                    df_F.loc[df_F['PAT_ID']==hit['mrn'],'PROBLEM_GROUP'] = key
                    df_F.loc[df_F['PAT_ID']==hit['mrn'],'PROBLEM_ID'] = ','.join(map(str,[x[1] for x in hit.matched_terms()]))
                    df_F.loc[df_F['PAT_ID']==hit['mrn'],'PROBLEMS']  = hit['content']
                else:
                    df_F.loc[df_F['PAT_ID']==hit['mrn'],'PROBLEM_GROUP'] = key + ',' + df_F.loc[df_F['PAT_ID']==hit['mrn'],'PROBLEM_GROUP'].values[0]
                    df_F.loc[df_F['PAT_ID']==hit['mrn'],'PROBLEM_ID'] = ','.join(map(str,[x[1] for x in hit.matched_terms()])) + ',' + df_F.loc[df_F['PAT_ID']==hit['mrn'],'PROBLEM_ID'].values[0]
                    df_F.loc[df_F['PAT_ID']==hit['mrn'],'PROBLEMS']  = hit['content'] + ',' + df_F.loc[df_F['PAT_ID']==hit['mrn'],'PROBLEMS'].values[0]

In [ ]:
len(df_F)

In [ ]:
# Debug
display(df_F[df_F['PAT_ID'].isin(['Z1256768','Z832424'])])

In [ ]:
df_F.to_csv(output+'/corrected_whoosh_data_withProblemList.csv')

# Step 4: Read Sheet 6 - Medication List

| Field         	| Field description             	|
|---------------	|-------------------------------	|
| GIVEN_MRN     	| MRN provided                  	|
| NEW_MRN       	| New MRN (if updated)          	|
| PAT_ID        	| Patient Identifier field      	|
| ORDER_MED_ID  	| Medication order identifier # 	|
| ORDERING_DATE 	| Date of order                 	|

In [ ]:
# Load Data with Problems attached
tmpfile = output+'/corrected_whoosh_data_withProblemList.csv'
df_F = pd.read_csv(tmpfile, index_col=False)

rows = df_F.PAT_ID[pd.isnull(df_F.PROBLEM_ID.values)]
print(len(rows))

df_6 = read_sheet(6)
#Visualize Sheet
df_6 = get_df_stats(df_6,['PAT_ID'])

# Sort by Latest date for problem list
df_6['ORDERING_DATE'] = pd.to_datetime(df_6.ORDERING_DATE)
#df_6 = df_6.sort_values(by='ORDERING_DATE',ascending=False)
#display(df_6.head(100))

In [ ]:
len(df_F)

In [ ]:
def convert_values_lower(in_val):
    if isinstance(in_val,str):
        return in_val.lower()
    else:
        out_val = [convert_values_lower(value) for value in in_val]
        return out_val
    
def convert_dict_lower(in_dict):
    out_dict = {}
    for key,val in in_dict.items():
        new_key = key.lower()
        new_val = convert_values_lower(val)
        out_dict[new_key] = new_val
    return out_dict

In [ ]:
from data.search_terms_v5 import get_meds
med_list = get_meds();

In [ ]:
med_list = convert_dict_lower(med_list)
print(len(med_list.keys()))

In [ ]:
med_list

In [ ]:
n_searchterms = list(itertools.chain(*med_list.values()))
print('Number of problems: ', len(n_searchterms))

In [ ]:
# Find number of patients that are in the Medication list
print("Number of patients in Master List: ")
print(len(df_F))
print("Number of patients in Medication List: ")
med_pats = [pat_id for pat_id in df_F.PAT_ID if pat_id in df_6.PAT_ID.values]
print(len(med_pats))

In [ ]:
df_F["MED_GROUP"] = None
df_F["MED_ID"] = None
df_F["MEDS"] = None
pat_ind = df_F.PAT_ID.isin(med_pats).values
pat_ind_neg =~pat_ind
df_F.loc[pat_ind_neg, 'MED_GROUP'] = 'no_meds'
df_F.loc[pat_ind, 'MED_GROUP'] = 'unknown_meds'
display(df_6.head(100))

In [ ]:
# Create schema for Med List search
schema = Schema(mrn=ID(stored=True), content=TEXT(stored=True))

# indexdir is a directory to store the search index
im = create_in(output+'/whoosh', schema)

writer = im.writer()

for ind in df_6.index:
    # add a document to search index that is keyed by MRN with the content being the problem list
    problem = df_6.loc[ind,'MEDICATION_NAME']
    route = df_6.loc[ind,'ROUTE']
    pat_id = df_6.loc[ind,'PAT_ID']
    #print(problem)
    #break
    if pd.isnull(route):
        #print(problem,type(problem))
        route = ''
    if pd.isnull(problem):
        #print(problem,type(problem))
        problem = ''
    problem = problem + ' ' + route
    writer.add_document(
    mrn=pat_id,
    content=problem
    )

In [ ]:
writer.commit()

In [ ]:
# Perform Query
with im.searcher() as searcher:
    # get all of the terms from the problem list and make term objects
    #terms = [Term("content", problem) for problem in itertools.chain.from_iterable(problem_list.values())]
    for key,value in med_list.items():
        terms = list()
        for problem in value:
            if isinstance(problem, str):
                words = problem.split(' ')
                if len(words) == 1:
                   # print(problem)
                    terms.append(Term("content",problem))
                else:
                    word_t  = list()
                    for word in words:
                        word_t.append(Term("content",word))
                    terms.append(And(word_t))
            else:
                #print(problem)
                terms.append(Term("content",problem[0]) & Term("content",problem[1]))
        # Generates a query that will match cancer or lymphoma or neuroblastoma or ... and so on
        query = Or(terms)
       # query = QueryParser("content", im.schema).parse(problem[0]+' '+problem[1])
        results = searcher.search(query, terms=True, limit=None)
        print("Group searched", key)
        print(results.estimated_length())
        for i,hit in enumerate(results):
            #print(i,hit['mrn'])
            if hit['mrn'] in med_pats:
               # print (hit['mrn'], ' ',df_F.loc[df_F['PAT_ID']==hit['mrn'],'PROBLEM_GROUP'].values)
               # print(hit['mrn'])
               # print(','.join(map(str,[x[1] for x in hit.matched_terms()])))
               # print(df_F.loc[df_F['PAT_ID']==hit['mrn'],'MED_ID'].values[0])
               # print(df_F.loc[df_F['PAT_ID']==hit['mrn'],'MED_GROUP'].values[0])
                if df_F.loc[df_F['PAT_ID']==hit['mrn'],'MED_GROUP'].values[0] == 'unknown_meds':
                    df_F.loc[df_F['PAT_ID']==hit['mrn'],'MED_GROUP'] = key
                    df_F.loc[df_F['PAT_ID']==hit['mrn'],'MED_ID'] = ','.join(map(str,[x[1] for x in hit.matched_terms()]))
                    df_F.loc[df_F['PAT_ID']==hit['mrn'],'MEDS']  = hit['content']
                else:
                    df_F.loc[df_F['PAT_ID']==hit['mrn'],'MED_GROUP'] = key + ',' + df_F.loc[df_F['PAT_ID']==hit['mrn'],'MED_GROUP'].values[0]
                   # df_F.loc[df_F['PAT_ID']==hit['mrn'],'MED_ID'] = ','.join(map(str,[x[1] for x in hit.matched_terms()])) + ',' + df_F.loc[df_F['PAT_ID']==hit['mrn'],'MED_ID'].values[0]
                   # df_F.loc[df_F['PAT_ID']==hit['mrn'],'MEDS']  = hit['content'] + ',' + df_F.loc[df_F['PAT_ID']==hit['mrn'],'MEDS'].values[0]

In [ ]:
df_meds = df_F[(df_F['MED_GROUP']!='unknown_meds') & (df_F['MED_GROUP']!='no_meds')]

In [ ]:
len(df_meds)

In [ ]:
display(df_F.head(100))

In [ ]:
df_F.to_csv(output+'/corrected_whoosh_data_withProblemList_withMedList.csv')

# Step 5: Read Sheet 3 - LDA

| Field             	| Field description                          	|
|-------------------	|--------------------------------------------	|
| GIVEN_MRN         	| MRN provided                               	|
| NEW_MRN           	| New MRN (if updated)                       	|
| PAT_ID            	| Patient Identifier field                   	|
| TYPE              	| Type of log                                	|
| SURGERY_DATE      	| Date of surgery                            	|
| CASE_SERVICE_NAME 	| Service name associated with surgical case 	|
| DESCRIPTION       	| Name/description of surgery                	|

In [ ]:
df_3 = read_sheet(3)
#Visualize Sheet
df_3 = get_df_stats(df_3,['PAT_ID'])
display(df_3.head(100))

In [ ]:
# Sort by Latest date for problem list
df_3['DATE_OF_ENTRY'] = pd.to_datetime(df_3.REMOVAL_DTTM)
df_3 = df_3.sort_values(by='DATE_OF_ENTRY',ascending=False)

In [ ]:
from data.search_terms_v5 import get_lda
lda_list = get_lda();
# Convert all terms to lower for Whoosh search
lda_list = convert_dict_lower(lda_list)
print(lda_list)

In [ ]:
# Find number of patients that are in the Medication list
print("Number of patients in Master List: ")
print(len(df_F))
print("Number of patients in LDA List: ")
lda_pats = [pat_id for pat_id in df_F.PAT_ID if pat_id in df_3.PAT_ID.values]
print(len(lda_pats))

In [ ]:
df_F["LDA_GROUP"] = None
df_F["LDA_ID"] = None
df_F["LDAS"] = None
pat_ind = df_F.PAT_ID.isin(lda_pats).values
pat_ind_neg =~pat_ind
df_F.loc[pat_ind_neg, 'LDA_GROUP'] = 'no_ldas'
df_F.loc[pat_ind, 'LDA_GROUP'] = 'unknown_ldas'
display(df_3.head(100))

In [ ]:
# Create schema for LDA List search
schema = Schema(mrn=ID(stored=True), content=TEXT(stored=True))

# indexdir is a directory to store the search index
il = create_in(output+'/whoosh', schema)

writer = il.writer()

for ind in df_3.index:
    # add a document to search index that is keyed by MRN with the content being the problem list
    problem = df_3.loc[ind,'DESCRIPTION']
    pat_id = df_3.loc[ind,'PAT_ID']
    if pd.isnull(problem):
        #print(problem,type(problem))
        problem = ''
    writer.add_document(
    mrn=pat_id,
    content=problem
    )

In [ ]:
writer.commit()

In [ ]:
# Perform Query
with il.searcher() as searcher:
    # get all of the terms from the problem list and make term objects
    #terms = [Term("content", problem) for problem in itertools.chain.from_iterable(problem_list.values())]
    for key,value in lda_list.items():
        terms = list()
        for problem in value:
            if isinstance(problem, str):
                words = problem.split(' ')
                if len(words) == 1:
                   # print(problem)
                    terms.append(Term("content",problem))
                else:
                    word_t  = list()
                    for word in words:
                        word_t.append(Term("content",word))
                    terms.append(And(word_t))
            else:
                #print(problem)
                terms.append(Term("content",problem[0]) & Term("content",problem[1]))
        # Generates a query that will match cancer or lymphoma or neuroblastoma or ... and so on
        query = Or(terms)
       # query = QueryParser("content", im.schema).parse(problem[0]+' '+problem[1])
        results = searcher.search(query, terms=True, limit=None)
        print("Group searched", key)
        print(results.estimated_length())
        for hit in results:
            if hit['mrn'] in lda_pats:
               # print (hit['mrn'], ' ',df_F.loc[df_F['PAT_ID']==hit['mrn'],'PROBLEM_GROUP'].values)
               # print(hit['mrn'])
               # print(','.join(map(str,[x[1] for x in hit.matched_terms()])))
               # print(df_F.loc[df_F['PAT_ID']==hit['mrn'],'LDA_ID'].values[0])
               # print(df_F.loc[df_F['PAT_ID']==hit['mrn'],'LDA_GROUP'].values[0])
                if df_F.loc[df_F['PAT_ID']==hit['mrn'],'LDA_GROUP'].values[0] == 'unknown_ldas':
                    df_F.loc[df_F['PAT_ID']==hit['mrn'],'LDA_GROUP'] = key
                    df_F.loc[df_F['PAT_ID']==hit['mrn'],'LDA_ID'] = ','.join(map(str,[x[1] for x in hit.matched_terms()]))
                    df_F.loc[df_F['PAT_ID']==hit['mrn'],'LDAS']  = hit['content']
                else:
                    df_F.loc[df_F['PAT_ID']==hit['mrn'],'LDA_GROUP'] = key + ',' + df_F.loc[df_F['PAT_ID']==hit['mrn'],'LDA_GROUP'].values[0]
                  #  df_F.loc[df_F['PAT_ID']==hit['mrn'],'LDA_ID'] = ','.join(map(str,[x[1] for x in hit.matched_terms()])) + ',' + df_F.loc[df_F['PAT_ID']==hit['mrn'],'LDA_ID'].values[0]
                  #  df_F.loc[df_F['PAT_ID']==hit['mrn'],'LDAS']  = hit['content'] + ',' + df_F.loc[df_F['PAT_ID']==hit['mrn'],'LDAS'].values[0]

In [ ]:
df_F.to_csv(output+'/corrected_whoosh_data_withProblemList_withMedList_withLDA.csv')

In [ ]:
infile  = output+'/corrected_whoosh_data_withProblemList_withMedList_withLDA.csv'
#infile  = output+'/corrected_whoosh_data.csv'
df_F = pd.read_csv(infile, index_col=False)

# Step 5: Read Sheet 9 - Surgeries

| Field             	| Field description                          	|
|-------------------	|--------------------------------------------	|
| GIVEN_MRN         	| MRN provided                               	|
| NEW_MRN           	| New MRN (if updated)                       	|
| PAT_ID            	| Patient Identifier field                   	|
| TYPE              	| Type of log                                	|
| SURGERY_DATE      	| Date of surgery                            	|
| CASE_SERVICE_NAME 	| Service name associated with surgical case 	|
| DESCRIPTION       	| Name/description of surgery                	|

In [ ]:
df_9 = read_sheet(9)
#Visualize Sheet
df_9 = get_df_stats(df_9,['PAT_ID'])

# Sort by Latest date for problem list
df_9['SURGERY_DATE'] = pd.to_datetime(df_9.SURGERY_DATE)

In [ ]:
display(df_9.head(100))

In [ ]:
# Find number of patients that are in the Surgery list
print("Number of patients in Master List: ")
print(len(df_F))
print("Number of patients in Surgery List: ")
surg_pats = [pat_id for pat_id in df_F.PAT_ID if pat_id in df_9.PAT_ID.values]
print(len(surg_pats))

In [ ]:
df_F["SURGERY"] = None
df_F["SURGERY_to_Imaging_days"] = 'no_date'
df_F['CT_SCAN_DATE'] = pd.to_datetime(df_F.CT_SCAN_DATE)
pat_ind = df_F.PAT_ID.isin(surg_pats).values
pat_ind_neg =~pat_ind
df_F.loc[pat_ind_neg, 'SURGERY'] = 'no_surgery'
df_F.loc[pat_ind, 'SURGERY'] = 'surgery'
display(df_F.head(100))

In [ ]:
for pat_id in surg_pats:
        #print(pat_id)
        scan_date = df_F.loc[df_F['PAT_ID']==pat_id, 'CT_SCAN_DATE']
        rows = df_9.loc[df_9['PAT_ID']==pat_id]
        rows = rows.sort_values(by='SURGERY_DATE',ascending=False)
        surgery_dates = rows.SURGERY_DATE.values
        for surgery_date in surgery_dates:
            #print('Scan date: ',scan_date)
            datediff = abs(scan_date - surgery_date).astype('timedelta64[D]').iloc[0]
            #print(datediff)
            if datediff > 30:
                df_F.loc[df_F['PAT_ID']==pat_id, 'SURGERY_to_Imaging_days'] = '> 30'
            else:
                df_F.loc[df_F['PAT_ID']==pat_id, 'SURGERY_to_Imaging_days'] = '<= 30'

In [ ]:
df_F.to_csv(output+'/corrected_whoosh_data_withProblemList_withMedList_withLDA_withSurgery.csv')

In [ ]:
display(df_F.head(100))

# Step 6: Read Sheet 10 - Surgical History

| Field             	| Field description                          	|
|-------------------	|--------------------------------------------	|
| GIVEN_MRN         	| MRN provided                               	|
| NEW_MRN           	| New MRN (if updated)                       	|
| PAT_ID            	| Patient Identifier field                   	|
| PROCEDURE         	| Surgery Performed                          	|
| SURGICAL_HX_DATE  	| Date of surgery                            	|

In [ ]:
# Load the Previously Saved file from epic_filter_1: 
tmpfile = output+'/corrected_whoosh_data_withProblemList_withMedList_withLDA_withSurgery.csv'
df_F = pd.read_csv(tmpfile, index_col=False)
display(df_F.head(100))

In [ ]:
df_10 = read_sheet(10)
#Visualize Sheet
df_10 = get_df_stats(df_10,['PAT_ID','SURGICAL_HX_DATE'])

# Sort by Latest date for problem list
df_10['SURGICAL_HX_DATE'] = pd.to_datetime(df_10.SURGICAL_HX_DATE,errors='coerce')
display(df_10.head(100))

In [ ]:
# Find number of patients that are in the Surgery History
print("Number of patients in Master List: ")
print(len(df_F))
print("Number of patients in Surgery History List: ")
surghx_pats = [pat_id for pat_id in df_F.PAT_ID if pat_id in df_10.PAT_ID.values]
print(len(surghx_pats))

In [ ]:
df_F['CT_SCAN_DATE'] = pd.to_datetime(df_F.CT_SCAN_DATE)
for pat_id in surghx_pats:
        #print(pat_id)
        rows = df_10.loc[df_10['PAT_ID']==pat_id]
        rows = rows.sort_values(by='SURGICAL_HX_DATE',ascending=False)
        surgery_dates = rows.SURGICAL_HX_DATE.values
        for surgery_date in surgery_dates:
            #print('Scan date: ',scan_date)
            datediff = abs(scan_date - surgery_date).astype('timedelta64[D]').iloc[0]
            #print(datediff)
            if datediff > 30:
                df_F.loc[df_F['PAT_ID']==pat_id, 'SURGERY_to_Imaging_days'] = '> 30'
            else:
                df_F.loc[df_F['PAT_ID']==pat_id, 'SURGERY_to_Imaging_days'] = '<= 30'
                break

In [ ]:
df_F.to_csv(output+'/corrected_whoosh_data_withProblemList_withMedList_withLDA_withSurgery_withSurgHX.csv')

# Step 7: Select Patients

Filter by Surgery, Problem_list and Meds

In [ ]:
infile  = output+'/corrected_whoosh_data_withProblemList_withMedList_withLDA_withSurgery_withSurgHX.csv'
#infile  = output+'/corrected_whoosh_data.csv'
df_F = pd.read_csv(infile, index_col=False)


In [ ]:
df_F['ValidPatient'] = 0
df_F.loc[df_F.PROBLEM_GROUP.isin(['unknown_problem','no_problem']) &
         df_F.MED_GROUP.isin(['unknown_meds','no_meds']) &
         df_F.LDA_GROUP.isin(['unknown_ldas','no_ldas']) &
         df_F.SURGERY_to_Imaging_days.isin(['no_date','> 30']), 'ValidPatient'] = 1

display(df_F.head(100))

In [ ]:
df_F = df_F.sort_values(by=['ValidPatient'],ascending=False)
validPats = df_F.PAT_ID[df_F.ValidPatient==1]
invalidPats = df_F.PAT_ID[df_F.ValidPatient==0]
print(len(validPats))
print(len(invalidPats))

In [ ]:
df_valid = df_F[df_F.ValidPatient==1]
df_valid.to_csv(output+'/corrected_whoosh_data_validpatients_v5.csv')